In [1]:
import os, pickle, sys
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import glob
from prettytable import PrettyTable
from tqdm import tqdm

In [2]:
t=None
all_ds = {}
all_acc = {}
allc = {}
all_metrics = {}
all_runs = {}
metric_names = ['grad_norm', 'snip', 'grasp', 'fisher', 'jacob_cov']
for fname,rname in [('../results_release/nasbench2/nb2_cf10_seed42_dlrandom_dlinfo1_initwnone_initbnone.p','CIFAR10')]:
    runs=[]
    f = open(fname,'rb')
    while(1):
        try:
            runs.append(pickle.load(f))
        except EOFError:
            break
    f.close()
    print(fname, len(runs))
    print(runs[0])
    
    all_runs[fname]=runs
    all_ds[fname] = {}
    metrics={}
    for k in metric_names:
        metrics[k] = []
    acc = []
    
    if t is None:
        hl=['Dataset']
        hl.extend(metric_names)
        t = PrettyTable(hl)
    
    for r in runs:
        for k,v in r['logmeasures'].items():
            if k in metrics:
                metrics[k].append(v)
        acc.append(r['testacc'])
        
    all_ds[fname]['metrics'] = metrics
    all_ds[fname]['acc'] = acc

    res = []
    crs = {}
    for k in hl:
        if k=='Dataset':
            continue
        v = metrics[k]
        cr = abs(stats.spearmanr(acc,v,nan_policy='omit').correlation)
        #print(f'{k} = {cr}')
        res.append(round(cr,3))
        crs[k]=cr
    
    ds = rname
    all_acc[ds]=acc
    allc[ds]=crs
    t.add_row([ds]+res)
      
    all_metrics[ds] = metrics
print(t)

../results_release/nasbench2/nb2_cf10_seed42_dlrandom_dlinfo1_initwnone_initbnone.p 15625
{'i': 0, 'arch': '|avg_pool_3x3~0|+|nor_conv_1x1~0|skip_connect~1|+|nor_conv_1x1~0|skip_connect~1|skip_connect~2|', 'logmeasures': {'grad_norm': 3.2280678749084473, 'snip': 5.514030933380127, 'grasp': -0.1634197235107422, 'fisher': 0.004466889891773462, 'jacob_cov': -151.3307806940146, 'plain': 0.028314922004938126, 'synflow_bn': 16.958625670825334, 'synflow': 1.9405668633541168e+17}, 'trainacc': 88.91866665852865, 'valacc': 81.98266665690103, 'testacc': 81.52}
+---------+-----------+-------+-------+--------+-----------+
| Dataset | grad_norm |  snip | grasp | fisher | jacob_cov |
+---------+-----------+-------+-------+--------+-----------+
| CIFAR10 |   0.594   | 0.596 | 0.514 |  0.36  |   0.731   |
+---------+-----------+-------+-------+--------+-----------+


In [3]:
for i in range(10):
    print (runs[i]['logmeasures']['synflow'])

1.9405668633541168e+17
1.1493810330209561e+33
7.464802349925823e+22
2127.9747457035596
8.071480574484623e+20
5.1972791685016835e+22
4.239559914155398e+31
2.367273749427031e+18
6.434034652398955e+26
2459.222137292069


In [4]:
fname='../mytest/nb2_cf10_add.p'
rname='CIFAR10'
runs=[]
f = open(fname,'rb')
while(1):
    try:
        runs.append(pickle.load(f))
    except EOFError:
        break
f.close()
print(fname, len(runs))
print(runs[0])

../mytest/nb2_cf10_add.p 15625
{'i': 0, 'arch': '|avg_pool_3x3~0|+|nor_conv_1x1~0|skip_connect~1|+|nor_conv_1x1~0|skip_connect~1|skip_connect~2|', 'logmeasures': {'synflow': 1.9405668535630934e+17}, 'trainacc': 88.91866665852865, 'valacc': 81.98266665690103, 'testacc': 81.52}


In [5]:
metrics={}
metrics['synflow']=[]
acc=[]

In [6]:
hl.extend(['synflow'])

In [7]:
for r in runs:
    for k,v in r['logmeasures'].items():
        metrics[k].append(v)
    acc.append(r['testacc'])

In [8]:
len(metrics['synflow'])

15625

In [15]:
cr = abs(stats.spearmanr(acc,metrics['synflow'],nan_policy='omit').correlation)
print(cr)

0.7367038620993901


In [10]:
acc

[81.52,
 90.51333333333332,
 81.83999999999999,
 83.88333333333334,
 84.49666666666667,
 87.78000000000002,
 88.64666666666666,
 88.77333333333333,
 88.81,
 84.11666666666666,
 10.0,
 87.64999999999999,
 10.0,
 87.83666666666666,
 88.3,
 56.19333333333333,
 89.90333333333332,
 86.19666666666667,
 87.44666666666667,
 84.35666666666667,
 86.92666666666666,
 89.14,
 85.79333333333334,
 86.14999999999999,
 81.62333333333332,
 83.87666666666667,
 86.94333333333334,
 86.55000000000001,
 82.7,
 84.46,
 88.51333333333334,
 87.14666666666666,
 89.07666666666665,
 88.14666666666666,
 88.22333333333334,
 87.67,
 87.39333333333333,
 86.55000000000001,
 88.49333333333334,
 86.70666666666666,
 71.41333333333334,
 85.72000000000001,
 90.64333333333333,
 88.37,
 75.18666666666667,
 86.47666666666667,
 86.88333333333333,
 86.71999999999998,
 82.67666666666668,
 87.78333333333335,
 89.62333333333333,
 85.27666666666666,
 90.29333333333334,
 34.196666666666665,
 87.28666666666668,
 84.14666666666666,
 90

In [11]:
res.append(round(cr,3))

In [12]:
res

[0.594, 0.596, 0.514, 0.36, 0.731, 0.737]

In [13]:
t = PrettyTable(hl)
t.add_row([rname]+res)
print(t)

+---------+-----------+-------+-------+--------+-----------+---------+
| Dataset | grad_norm |  snip | grasp | fisher | jacob_cov | synflow |
+---------+-----------+-------+-------+--------+-----------+---------+
| CIFAR10 |   0.594   | 0.596 | 0.514 |  0.36  |   0.731   |  0.737  |
+---------+-----------+-------+-------+--------+-----------+---------+


In [14]:
fname='../nb2_cf10_seed42_dlrandom_dlinfo1_initwnone_initbnone.p'
rname='CIFAR10'
runs=[]
f = open(fname,'rb')
while(1):
    try:
        runs.append(pickle.load(f))
    except EOFError:
        break
f.close()
print(fname, len(runs))
for i in runs:
    print(i['logmeasures']['synflow'])

../nb2_cf10_seed42_dlrandom_dlinfo1_initwnone_initbnone.p 10
1.9653066517198026e+17
1.0532509569211224e+33
7.250251707593463e+22
2448.7573842802194
7.477393253925509e+20
5.5718184552029185e+22
4.070820358843067e+31
2.2594474650100708e+18
6.203224266732338e+26
2574.158019202686
